In [63]:
import pandas as pd
import pytorch_lightning as pl
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader

from maderapp.model import TimberMobileNet
from maderapp.validator import validator
from maderapp.data.data_inference import MaderappDatasetInference

model_path = "maderapp-epoch=247-val_loss=0.05.ckpt"
model_name = "MobileNet"
num_classes = 27

In [64]:
model = TimberMobileNet(num_classes)
model = model.load_from_checkpoint(
    model_path,
    num_classes=num_classes,
)
model.eval()

trainer = pl.Trainer(
    gpus=1,
)

metadata = pd.read_csv("../metadata-v3.csv", header=None)
class_names = sorted(metadata.iloc[:, 1].value_counts().index)
class_names2ids = {j: i for i, j in enumerate(class_names)}
class_ids2names = {j: i for i, j in class_names2ids.items()}


/home/ubuntu/anaconda3/envs/maderapp/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number no trainable parameters: 2223872 
Number trainable parameters: 34587
Number no trainable parameters: 2223872 
Number trainable parameters: 34587


In [65]:
data_path = "/home/ubuntu/data_maderapp/validation/"
metadata = [str(path) for path in list(Path(data_path).glob("*.jpg"))]
metadata[:5]

['/home/ubuntu/data_maderapp/validation/2382.jpg',
 '/home/ubuntu/data_maderapp/validation/1739.jpg',
 '/home/ubuntu/data_maderapp/validation/786.jpg',
 '/home/ubuntu/data_maderapp/validation/1720.jpg',
 '/home/ubuntu/data_maderapp/validation/2041.jpg']

In [66]:
transformation = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],),
        ToTensorV2(),
    ]
)

In [67]:
import torch
x = torch.randn(1,3,224,224)
y_pred = model(x)
class_ids2names[int(y_pred.argmax())]

/home/ubuntu/maderapp/maderapp/model/timber_clasification_mobileNet.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(out)


'OOC'

In [68]:
metadata = ["/home/ubuntu/data_maderapp/validation-v3/cumala.jpg", "/home/ubuntu/data_maderapp/validation-v3/palo_verde.jpg"]
ds = MaderappDatasetInference(annotations_file=metadata, transform=transformation)
dl = DataLoader(ds, batch_size=2, shuffle=False, num_workers=4)

x, y = next(iter(dl))

y_pred = model(x)
class_pred = [class_ids2names[int(idx)] for idx in y_pred.argmax(dim=1)]

print(class_pred)

['Cumala', 'Palo_Verde']


In [69]:
y_pred.max(dim=1)

torch.return_types.max(
values=tensor([0.7878, 0.9989], grad_fn=<MaxBackward0>),
indices=tensor([ 8, 17]))

In [73]:
x.shape

torch.Size([2, 3, 224, 224])

In [78]:
model1 = torch.jit.load("/home/ubuntu/data_maderapp/mobile_models/opt_trace_mobileNet-v3.pt")
y_pred = model1(x)
class_pred = [class_ids2names[int(idx)] for idx in y_pred.argmax(dim=1)]
y_pred.max(dim=1)

torch.return_types.max(
values=tensor([0.0796, 0.0720]),
indices=tensor([ 6, 11]))

torch.return_types.max(
values=tensor([0.0796, 0.0720]),
indices=tensor([ 6, 11]))